In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import gc #garbage collector

# clear cuda memory and collect garbage
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: ',device)

x=np.load('scaled_spec_resampled_array.npy')
y=np.load('labels_array.npy')-1
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])

print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Device:  cuda:0
(1754, 1, 2048, 80) (1754,)


In [2]:
resnet18 = models.resnet18(pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

resnet18 = resnet18.to(device)

#freeze every layer except the first conv1 layer and the last fc layer, to adapt the model to our data
for name, param in resnet18.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

def train(model, train_loader, test_loader, criterion, optimizer, epochs):
    model.train()
    running_loss = 0.0
    epoch_bar = tqdm(range(epochs), position=0)
    for epoch in epoch_bar:
        batch_bar=tqdm(enumerate(train_loader, 0), total=len(train_loader), position=1, leave=False)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_bar.set_description('Train loss: %.3f' % (loss.item()))
        train_loss = running_loss / len(train_loader)
        epoch_bar.set_description('Train loss: %.3f' % train_loss)
        val_loss, val_acc = test(model, test_loader, criterion)
        print('Epoch: %d, Train Loss: %.3f, Val Loss: %.3f, Val Acc: %.3f' % (epoch, train_loss, val_loss, val_acc))
    return model


In [4]:
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet18=train(resnet18, train_loader,test_loader, criterion, optimizer, 100)

torch.save(resnet18.state_dict(), 'resnet18.pth')

Train loss: 1.678:   1%|          | 1/100 [00:25<41:21, 25.07s/it]

Epoch: 0, Train Loss: 1.678, Val Loss: 1.766, Val Acc: 0.225


Train loss: 3.183:   2%|▏         | 2/100 [00:33<25:01, 15.32s/it]

Epoch: 1, Train Loss: 3.183, Val Loss: 1.260, Val Acc: 0.462


Train loss: 4.450:   3%|▎         | 3/100 [00:42<19:51, 12.28s/it]

...

Epoch: 97, Train Loss: 70.651, Val Loss: 0.529, Val Acc: 0.801


Train loss: 71.090:  99%|█████████▉| 99/100 [16:18<00:09,  9.58s/it]

Epoch: 98, Train Loss: 71.090, Val Loss: 0.502, Val Acc: 0.812


Train loss: 71.538: 100%|██████████| 100/100 [16:27<00:00,  9.88s/it]


Epoch: 99, Train Loss: 71.538, Val Loss: 0.547, Val Acc: 0.792
